In [2]:
!pip install langchain_community
!pip install langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1

In [3]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

import glob
import os
import shutil
from dotenv import load_dotenv


import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

from langchain_community.document_loaders import (
    DirectoryLoader,
    PyPDFLoader,
    TextLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from huggingface_hub import login, HfApi



from langchain_core.documents import Document
from langchain_chroma import Chroma

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma 

2026-02-17 10:16:37.993436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771323398.169094      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771323398.222760      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771323398.686217      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771323398.686263      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771323398.686266      55 computation_placer.cc:177] computation placer alr

In [4]:
# ==============================
# Load secrets from Kaggle
# ==============================
from kaggle_secrets import UserSecretsClient

# Initialize the client
user_secrets = UserSecretsClient()

# Fetch your secrets
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")


# Optional: check if they loaded correctly (do NOT print tokens in public!)
assert HF_TOKEN is not None, "HF_TOKEN not found!"



In [7]:
# Hugging Face login
from huggingface_hub import login
login(token=HF_TOKEN)



LOCAL_DB_PATH = "chroma_db"

In [8]:
print("Loading documents...")

folders = glob.glob("/kaggle/input/datasets/mdmarufmullah/raw-data/file/*")
documents = []

for folder in folders:
    doc_type = os.path.basename(folder)

    pdf_loader = DirectoryLoader(
        folder,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )

    md_loader = DirectoryLoader(
        folder,
        glob="**/*.md",
        loader_cls=TextLoader,
        loader_kwargs={"encoding": "utf-8"}
    )

    for loader in [pdf_loader, md_loader]:
        try:
            folder_docs = loader.load()
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                documents.append(doc)
        except Exception as e:
            print(f"Skipping some files: {e}")

print(f"Loaded {len(documents)} documents.")

if not documents:
    print("No documents found.")
    

# ==============================
# Split
# ==============================
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks.")

# ==============================
# Create Embeddings
# ==============================
print("Creating embeddings...")

from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(LOCAL_DB_PATH):
    Chroma(persist_directory=LOCAL_DB_PATH, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=LOCAL_DB_PATH)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


Loading documents...


ERROR:langchain_community.document_loaders.directory:Error loading file /kaggle/input/datasets/mdmarufmullah/raw-data/file/chat/chat50.pdf


Skipping some files: 'bbox'
Loaded 6 documents.
Created 25 chunks.
Creating embeddings...


/tmp/ipykernel_55/2094448380.py:55: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vectorstore created with 25 documents


In [15]:
# ==============================
# Load Fine-Tuned RAG Model
# ==============================
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import gradio as gr

RAG_MODEL_ID = "marufmullah50/Ai-Twin-2026-02-16_21.13.33"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Loading fine-tuned RAG model...")
tokenizer = AutoTokenizer.from_pretrained(RAG_MODEL_ID, use_auth_token=os.environ.get("HF_TOKEN"))
rag_model = AutoModelForCausalLM.from_pretrained(
    RAG_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    trust_remote_code=True,
    use_auth_token=os.environ.get("HF_TOKEN")
)

if DEVICE == "cuda":
    rag_model = rag_model.to(DEVICE)

rag_model.eval()
print("RAG model loaded!")

# ==============================
# Chat Function
# ==============================
def chat(message, history):
    if history is None:
        history = []

    history.append({"role": "user", "content": message})

    # Retrieve relevant context — 15 chunks for richer coverage
    docs = vectorstore.similarity_search(message, k=15)
    context = "\n\n".join([doc.page_content for doc in docs])

    conversation = f"""You are the AI Twin of Md. Maruf Mullah. Follow these rules strictly:
1. Answer ONLY from the provided context below — do not guess or fabricate information.
2. If the answer is not found in the context, say: "I don't have information about that."
3. Respond exclusively in clear, grammatically correct English — no gibberish, symbols, or mixed languages.
4. Speak in first person as Maruf Mullah.
5. Give detailed, thorough answers. Expand on points with specifics from the context — do not give short or vague replies.

Context:
{context}

"""

    # Add history in Gemma chat format
    for msg in history:
        if msg["role"] == "user":
            conversation += f"<start_of_turn>user\n{msg['content']}<end_of_turn>\n"
        else:
            conversation += f"<start_of_turn>model\n{msg['content']}<end_of_turn>\n"

    conversation += "<start_of_turn>model\n"

    inputs = tokenizer(
        conversation,
        return_tensors="pt",
        truncation=True,
        max_length=4096  # bumped up to fit larger context
    )
    inputs = {k: v.to(rag_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = rag_model.generate(
            **inputs,
            max_new_tokens=600,       # was 300, now allows longer answers
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,   # prevents the model from looping/repeating itself on long outputs
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the last model turn
    if "<start_of_turn>model" in full_output:
        response = full_output.split("<start_of_turn>model")[-1].strip()
    else:
        response = full_output[len(conversation):].strip()

    # Clean up any trailing turn markers
    for stop_token in ["<end_of_turn>", "<start_of_turn>"]:
        if stop_token in response:
            response = response.split(stop_token)[0].strip()

    history.append({"role": "assistant", "content": response})
    return history


# ==============================
# Gradio UI
# ==============================
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 AI Twin Chatbot — Md. Maruf Mullah")

    chatbot = gr.Chatbot(height=500, type="messages")
    user_input = gr.Textbox(
        lines=2,
        placeholder="Ask me anything about Maruf..."
    )
    send_btn = gr.Button("Send")

    send_btn.click(
        chat,
        inputs=[user_input, chatbot],
        outputs=chatbot
    )
    user_input.submit(
        chat,
        inputs=[user_input, chatbot],
        outputs=chatbot
    )

demo.launch(server_name="0.0.0.0", share=True)

Loading fine-tuned RAG model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RAG model loaded!
* Running on local URL:  http://0.0.0.0:7864
* Running on public URL: https://5f391da9ef392899f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
